In [63]:
import tensorflow as tf
import keras
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import TensorBoard
import os
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam

In [64]:
path = "gesture_data"

data = []
labels = []

for file_name in os.listdir(path):
    if file_name .endswith(".csv"):
        label = file_name.split("_")[0]
        file_path = os.path.join(path, file_name)

        df = pd.read_csv(file_path)
        data.append(df.values)
        labels.append(label)

In [65]:
np.array(data).shape, np.array(labels).shape

((31, 30, 1629), (31,))

In [66]:
X = np.array(data)
y = np.array(labels)

In [67]:
label_encoder = LabelEncoder()
y_encoded = to_categorical(label_encoder.fit_transform(labels)) 

In [68]:
model = keras.Sequential([
    Input(shape=(30, 1629)),
    LSTM(64, return_sequences=True, activation='relu'),
    LSTM(128, return_sequences=True, activation='relu'),
    LSTM(64, return_sequences=False, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(4, activation='softmax')
])

In [69]:
adam = Adam(learning_rate=0.0001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [70]:
from keras.callbacks import EarlyStopping

callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True,)
history = model.fit(
    X,y_encoded,
    epochs=100,
    validation_split=0.2,
    batch_size=16,
    callbacks=callback
)

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.2431 - loss: 1.3888 - val_accuracy: 0.2857 - val_loss: 1.3705
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1806 - loss: 1.3869 - val_accuracy: 0.2857 - val_loss: 1.3745
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.2222 - loss: 1.3763 - val_accuracy: 0.2857 - val_loss: 1.3707
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.4306 - loss: 1.3704 - val_accuracy: 0.2857 - val_loss: 1.3716
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6736 - loss: 1.3660 - val_accuracy: 0.2857 - val_loss: 1.3682
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5764 - loss: 1.3584 - val_accuracy: 0.2857 - val_loss: 1.3633
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5556 - loss: 1.3487 - val_accuracy: 0.2857 - val_loss: 1.3642
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.5069 - loss: 1.3378 - val_accuracy: 0.2857 - val_loss: 1

In [73]:
model.save('gesture_classifier.h5')

In [74]:
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class Mapping:")
for class_name, encoded_value in class_mapping.items():
    print(f"{class_name}: {encoded_value}")

Class Mapping:
call: 0
hi: 1
same: 2
thanks: 3


In [75]:
file_name = "class_mapping.txt"
with open(file_name, "w") as file:
    file.write("Class Mapping:\n")
    for class_name, encoded_value in class_mapping.items():
        file.write(f"{class_name}: {encoded_value}\n")

print(f"Class mapping saved to {file_name}")

Class mapping saved to class_mapping.txt
